
## RankNet as described in [1].
[1] http://icml.cc/2015/wp-content/uploads/2015/06/icml_ranking.pdf

[2] https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf

In [8]:
import numpy as np
import torch
import torch.nn as nn


In [2]:
#binary case 1--> relevant 0--> no relevant

In [139]:
# Data.
input_dim = 10
n_docs = 20
n_rel2 = 5
n_rel1 = 5
n_irr = n_docs - n_rel2 - n_rel1
y=np.array([2]*n_rel2+[1]*n_rel1+[0]*n_irr)
unique, unique_index = np.unique(y, return_index=True)
unique=np.flip(unique,axis=0).copy()
unique = torch.from_numpy(unique)
unique_index=np.flip(unique_index,axis=0).copy()
unique_index = torch.from_numpy(unique_index)
unique = unique.to(device)
unique_index = unique_index.to(device)
doc_features = np.random.randn(n_docs, input_dim)

# Document scores.
docs = torch.from_numpy(np.array(doc_features, dtype = "float32"))
docs = docs.to(device)


tensor([10,  5,  5])

In [101]:
torch.diff(y_t)

AttributeError: module 'torch' has no attribute 'diff'

In [93]:
# Model.
L1=128//2
L2=64//2
L3=32//2
model = torch.nn.Sequential(
    nn.Linear(input_dim, L1),
    nn.ReLU(),
    nn.Linear(L1, L2),
    nn.ReLU(),
    nn.Linear(L2, L3),
    nn.ReLU(),
    nn.Linear(L3, 1))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)
doc_scores = model(docs)
doc_scores[doc_scores<0]=0
#print(doc_scores)
ndcg_before=ndcg_at_k(doc_scores.tolist())
print(ndcg_before)

cpu
0.0


In [142]:

print(loss)

tensor([0.])


In [121]:
n_epoch = 1000
loss = torch.zeros(1)
loss = loss.to(device)
for epoch in range(n_epoch):  # loop over the dataset multiple times
    running_loss = 0.0
    # forward
    doc_scores = model(docs)
    loss = torch.zeros(1)
    for idx, ui in enumerate(unique_index[1:]):
        o_ij=doc_scores[:n_rel2]-doc_scores[n_rel2:].view(-1)
        c_ij = o_ij - torch.log(1.0+torch.exp(o_ij))
        loss+=c_ij.sum()
    # backward
    model.zero_grad()
    loss.backward()

    # Update model weights.
    lr = 0.001
    with torch.no_grad():
        for param in model.parameters():
            param += lr * param.grad

    # print statistics
    if epoch % (n_epoch//10) ==0:
        print(f"{epoch} : {loss}")
          

print('Finished Training')
doc_scores = model(docs)
print(doc_scores)
doc_scores[doc_scores<0]=0
print(doc_scores)
ndcg_after=ndcg_at_k(doc_scores.tolist())
print(ndcg_after)

0 : -125.90130615234375
100 : -0.5084235668182373
200 : -0.19798851013183594
300 : -0.11553668975830078
400 : -0.07860898971557617
500 : -0.058447837829589844
600 : -0.04600095748901367
700 : -0.03762960433959961
800 : -0.03165435791015625
900 : -0.027197837829589844
Finished Training
tensor([[ -2.4126],
        [ -0.6878],
        [ -0.9928],
        [ -0.9582],
        [ -2.1191],
        [ -9.0947],
        [ -9.3157],
        [ -9.1057],
        [-10.7002],
        [-15.1604],
        [ -9.2678],
        [-17.3323],
        [-13.8892],
        [-18.1695],
        [ -9.6713],
        [ -9.7246],
        [-20.3776],
        [-12.4374],
        [ -8.3814],
        [ -9.2085]], grad_fn=<ThAddmmBackward>)
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_

tensor([[ 7.4749],
        [ 9.6173],
        [10.6030],
        [ 8.4345],
        [ 6.0750],
        [ 2.2924],
        [ 0.8137],
        [ 1.8168],
        [ 0.0000],
        [ 0.0000],
        [ 2.3252],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 2.6281],
        [ 2.2651],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000]], grad_fn=<PutBackward>)

In [90]:
def dcg_at_k(r, k=5, method=1):
        """Score is discounted cumulative gain (dcg)
        Relevance is positive real values.  Can use binary
        as the previous methods.
        Example from
        http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
        >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
        >>> dcg_at_k(r, 1)
        3.0
        >>> dcg_at_k(r, 1, method=1)
        3.0
        >>> dcg_at_k(r, 2)
        5.0
        >>> dcg_at_k(r, 2, method=1)
        4.2618595071429155
        >>> dcg_at_k(r, 10)
        9.6051177391888114
        >>> dcg_at_k(r, 11)
        9.6051177391888114
        Args:
            r: Relevance scores (list or numpy) in rank order
                (first element is the first item)
            k: Number of results to consider
            method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                    If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
        Returns:
            Discounted cumulative gain
        """
        r = np.asfarray(r)[:k]
        if r.size > 0:
            if method == 0:
                return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
            elif method == 1:
                return float(np.sum(r / np.log2(np.arange(2, r.size + 2))))
            else:
                raise ValueError('method must be 0 or 1.')
        return 0.
def ndcg_at_k(r, k=5, method=1):
        """Score is normalized discounted cumulative gain (ndcg)
        Relevance is positive real values.  Can use binary
        as the previous methods.
        Example from
        http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
        >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
        >>> ndcg_at_k(r, 1)
        1.0
        >>> r = [2, 1, 2, 0]
        >>> ndcg_at_k(r, 4)
        0.9203032077642922
        >>> ndcg_at_k(r, 4, method=1)
        0.96519546960144276
        >>> ndcg_at_k([0], 1)
        0.0
        >>> ndcg_at_k([1], 2)
        1.0
        Args:
            r: Relevance scores (list or numpy) in rank order
                (first element is the first item)
            k: Number of results to consider
            method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                    If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
        Returns:
            Normalized discounted cumulative gain
        """
        dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
        if not dcg_max:
            return 0.
        return dcg_at_k(r, k, method) / dcg_max